In [50]:
using IntervalArithmetic
using LaTeXStrings
using Plots

In [51]:
include("./method4/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")

print()

In [52]:
function laTeXMatrix(
        pattern,
        size::Tuple{Integer, Integer})::String
    m, n = size

    mat = "\\begin{bmatrix}"
    for i = 1:m, j = 1:n
        mat *= pattern(i, j)
        if j == n
            if i != m
                mat *= " \\\\ "
            end
        else
            mat *= " & "
        end
    end
    mat *= "\\end{bmatrix}"

    return mat
end

laTeXMatrix (generic function with 1 method)

In [53]:
function method4(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    _, n = size(A₁)

    results = solveApproximationLP_m4.([A₁, A₂])
    for k = eachindex(results)
        wₖᴸ⁻ = results[k].wₖᴸ⁻; wₖᵁ⁻ = results[k].wₖᵁ⁻
        wₖᴸ⁺ = results[k].wₖᴸ⁺; wₖᵁ⁺ = results[k].wₖᵁ⁺
        A = fill((1..1, 1..1), (n, n))
        for i = 1:n, j = 1:n
            if i == j continue end
            aᵢⱼᴸ⁻ = wₖᴸ⁻[i] / wₖᵁ⁻[j]
            aᵢⱼᵁ⁻ = wₖᵁ⁻[i] / wₖᴸ⁻[j]
            aᵢⱼᴸ⁺ = wₖᴸ⁺[i] / wₖᵁ⁺[j]
            aᵢⱼᵁ⁺ = wₖᵁ⁺[i] / wₖᴸ⁺[j]
    
            aᵢⱼᴸ⁻ = correctPrecisionLoss(aᵢⱼᴸ⁻, aᵢⱼᴸ⁺)
            aᵢⱼᵁ⁻ = correctPrecisionLoss(aᵢⱼᵁ⁻, aᵢⱼᴸ⁻)
            aᵢⱼᵁ⁺ = correctPrecisionLoss(aᵢⱼᵁ⁺, aᵢⱼᵁ⁻)
            aᵢⱼᵁ⁺ = correctPrecisionLoss(aᵢⱼᵁ⁺, aᵢⱼᴸ⁺)
    
            if aᵢⱼᴸ⁻ > aᵢⱼᵁ⁻
                A[i, j] = (emptyinterval(), aᵢⱼᴸ⁺..aᵢⱼᵁ⁺)
            else
                A[i, j] = (aᵢⱼᴸ⁻..aᵢⱼᵁ⁻, aᵢⱼᴸ⁺..aᵢⱼᵁ⁺)
            end
        end
        println("\\tilde{A}_$k = $(twofoldIntervalMatrixLaTeXString(A))"); println()
    end

    A = generateTwofoldIntervalMatrix_m4(results)
    println("A = $(twofoldIntervalMatrixLaTeXString(A))"); println()

    result = solveCancellingLP_m4(A)
    resultₛₜᵣ = cancellingLPResultLaTeXString_m4(result)
    println("W = $(resultₛₜᵣ.W)"); println()
    println("""
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{L}-}", (n, 1))) = $(resultₛₜᵣ.wᴸ⁻), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{U}-}", (n, 1))) = $(resultₛₜᵣ.wᵁ⁻), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{L}+}", (n, 1))) = $(resultₛₜᵣ.wᴸ⁺), ~~
        $(laTeXMatrix((i,j) -> "w_$i^{\\text{U}+}", (n, 1))) = $(resultₛₜᵣ.wᵁ⁺)
        """); println()
    println("""
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{L}", (n, 1))) = $(resultₛₜᵣ.εᴸ), ~~
        $(laTeXMatrix((i,j) -> "\\varepsilon_$i^\\text{U}", (n, 1))) = $(resultₛₜᵣ.εᵁ)
        """); println()

    Â = updatePCM_m4(A, result)

    displayCancellingLPResultLaTeXString_m4(result)

    return Â
end

method4 (generic function with 1 method)

# example 1

In [59]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
A₂ = A₁

println("A_1 = A_2 = $(intervalMatrixLaTeXString(A₁))"); println()

Â = method4(A₁, A₂)

println("\\mathscr{A}^4 = $(twofoldIntervalMatrixLaTeXString(Â))")

L"$W = \begin{bmatrix}\left[ 0.235, 0.314 \right] \\ \left[ 0.157, 0.235 \right] \\ \left[ 0.235, 0.353 \right] \\ \left[ 0.235, 0.235 \right]\end{bmatrix}$"

L"$w^{\text{L}-} = \begin{bmatrix} 0.235  \\  0.157  \\  0.235  \\  0.235 \end{bmatrix}, ~~
w^{\text{U}-} = \begin{bmatrix} 0.314  \\  0.235  \\  0.353  \\  0.235 \end{bmatrix}, ~~
w^{\text{L}+} = \begin{bmatrix} 0.235  \\  0.157  \\  0.235  \\  0.235 \end{bmatrix}, ~~
w^{\text{U}+} = \begin{bmatrix} 0.314  \\  0.235  \\  0.353  \\  0.235 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}
$"

A_1 = A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 2.0 \right] & \left[ 0.667, 1.333 \right] & \left[ 1.0, 1.333 \right] \\ \left[ 0.5, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.444, 1.0 \right] & \left[ 0.667, 1.0 \right] \\ \left[ 0.75, 1.5 \right] & \left[ 1.0, 2.25 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.5 \right] \\ \left[ 0.75, 1.0 \right] & \left[ 1.0, 1.5 \right] & \left[ 0.667, 1.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}

\tilde{A}_1 = \begin{bmatrix} \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 1.0, \left[ 1.0, 2.0 \right], 2.0 \right] & \left[ 0.67, \left[ 0.67, 1.33 \right], 1.33 \right] & \left[ 1.0, \left[ 1.0, 1.33 \right], 1.33 \right] \\ \left[ 0.5, \left[ 0.5, 1.0 \right], 1.0 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 0.44, \left[ 0.44, 1.0 \right], 1.0 \right] & \left[ 0.67, \left[ 0.67, 1.0 \right], 1.0 \right] \\ \left[ 0.75, \left[ 0.75, 1.5 \right], 1.5 \right] & \left[ 1.0, \left[ 1.0

# example 2

In [55]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4/5)..(A₁[i,j].hi * 5/4)
end

Â = method4(A₁, A₂)

println("\\mathscr{A}^4 = $(twofoldIntervalMatrixLaTeXString(Â))")

L"$W = \begin{bmatrix}\left[ 0.206, 0.343 \right] \\ \left[ 0.137, 0.258 \right] \\ \left[ 0.206, 0.386 \right] \\ \left[ 0.206, 0.258 \right]\end{bmatrix}$"

L"$w^{\text{L}-} = \begin{bmatrix} 0.206  \\  0.137  \\  0.206  \\  0.206 \end{bmatrix}, ~~
w^{\text{U}-} = \begin{bmatrix} 0.275  \\  0.206  \\  0.309  \\  0.206 \end{bmatrix}, ~~
w^{\text{L}+} = \begin{bmatrix} 0.206  \\  0.137  \\  0.206  \\  0.206 \end{bmatrix}, ~~
w^{\text{U}+} = \begin{bmatrix} 0.275  \\  0.206  \\  0.309  \\  0.206 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}
$"

\tilde{A}_1 = \begin{bmatrix} \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 1.0, \left[ 1.0, 2.0 \right], 2.0 \right] & \left[ 0.67, \left[ 0.67, 1.33 \right], 1.33 \right] & \left[ 1.0, \left[ 1.0, 1.33 \right], 1.33 \right] \\ \left[ 0.5, \left[ 0.5, 1.0 \right], 1.0 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 0.44, \left[ 0.44, 1.0 \right], 1.0 \right] & \left[ 0.67, \left[ 0.67, 1.0 \right], 1.0 \right] \\ \left[ 0.75, \left[ 0.75, 1.5 \right], 1.5 \right] & \left[ 1.0, \left[ 1.0, 2.25 \right], 2.25 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 1.0, \left[ 1.0, 1.5 \right], 1.5 \right] \\ \left[ 0.75, \left[ 0.75, 1.0 \right], 1.0 \right] & \left[ 1.0, \left[ 1.0, 1.5 \right], 1.5 \right] & \left[ 0.67, \left[ 0.67, 1.0 \right], 1.0 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] \end{bmatrix}

\tilde{A}_2 = \begin{bmatrix} \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 0.8, \left[ 0.8, 2.5 \right], 2.

<!-- ともに弱い整合性を持ち， $A_{1ij} \subseteq A_{2ij}$

```
A₁ = [
    1..1 2..5 2..4 2..3
    (1/5)..(1/2) 1..1 (1/5)..4 (1/5)..3
    (1/4)..(1/2) (1/4)..5 1..1 (1/4)..3
    (1/3)..(1/2) (1/3)..5 (1/3)..4 1..1
]
A₂ = fill(1..1, size(A₁))
_, n = size(A₁)
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 1/1.1)..(A₁[i,j].hi * 1.1)
end

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

display(isConsistentIntervalPCM(A₁))
display(isWeaklyConsistentIntervalPCM(A₁))
display(isConsistentIntervalPCM(A₂))
display(isWeaklyConsistentIntervalPCM(A₂))

plotAll(A₁, A₂)
``` -->

# example 3

In [56]:
A = [
    1 2 4 4
    1/2 1 2 2
    1/4 1/2 1 1
    1/4 1/2 1 1
]

A₁ = randamizedIntervalPCM(A, 1234, 3e-2)
A₂ = randamizedIntervalPCM(A, 134, 3e-2)

Â = method4(A₁, A₂)

println("\\mathscr{A}^4 = $(twofoldIntervalMatrixLaTeXString(Â))")

L"$W = \begin{bmatrix}\left[ 0.5, 0.501 \right] \\ \left[ 0.247, 0.251 \right] \\ \left[ 0.123, 0.127 \right] \\ \left[ 0.123, 0.126 \right]\end{bmatrix}$"

L"$w^{\text{L}-} = \begin{bmatrix} 0.5  \\  0.247  \\  0.123  \\  0.123 \end{bmatrix}, ~~
w^{\text{U}-} = \begin{bmatrix} 0.5  \\  0.251  \\  0.126  \\  0.126 \end{bmatrix}, ~~
w^{\text{L}+} = \begin{bmatrix} 0.5  \\  0.247  \\  0.123  \\  0.123 \end{bmatrix}, ~~
w^{\text{U}+} = \begin{bmatrix} 0.5  \\  0.251  \\  0.126  \\  0.126 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}
$"

\tilde{A}_1 = \begin{bmatrix} \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 1.98, \left[ 1.98, 2.03 \right], 2.04 \right] & \left[ 3.94, \left[ 3.97, 4.07 \right], 4.11 \right] & \left[ 3.96, \left[ 3.99, 4.05 \right], 4.07 \right] \\ \left[ 0.49, \left[ 0.49, 0.5 \right], 0.5 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 1.94, \left[ 1.96, 2.05 \right], 2.07 \right] & \left[ 1.95, \left[ 1.97, 2.04 \right], 2.05 \right] \\ \left[ 0.24, \left[ 0.25, 0.25 \right], 0.25 \right] & \left[ 0.48, \left[ 0.49, 0.51 \right], 0.51 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 0.97, \left[ 0.98, 1.01 \right], 1.03 \right] \\ \left[ 0.25, \left[ 0.25, 0.25 \right], 0.25 \right] & \left[ 0.49, \left[ 0.49, 0.51 \right], 0.51 \right] & \left[ 0.97, \left[ 0.99, 1.02 \right], 1.03 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] \end{bmatrix}

\tilde{A}_2 = \begin{bmatrix} \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 2.0,

# example 4

In [57]:
A = [
    1 2 3 10
    1/2 1 3 8
    1/3 1/3 1 3
    1/10 1/8 1/3 1
]

A₁ = randamizedIntervalPCM(A, 124, 4e-1)
A₂ = randamizedIntervalPCM(A, 411, 4e-1)

Â = method4(A₁, A₂)

println("\\mathscr{A}^4 = $(twofoldIntervalMatrixLaTeXString(Â))")

L"$W = \begin{bmatrix}\left[ 0.467, 0.517 \right] \\ \left[ 0.274, 0.326 \right] \\ \left[ 0.127, 0.205 \right] \\ \left[ 0.038, 0.046 \right]\end{bmatrix}$"

L"$w^{\text{L}-} = \begin{bmatrix} 0.461  \\  0.232  \\  0.107  \\  0.038 \end{bmatrix}, ~~
w^{\text{U}-} = \begin{bmatrix} 0.517  \\  0.323  \\  0.131  \\  0.046 \end{bmatrix}, ~~
w^{\text{L}+} = \begin{bmatrix} 0.461  \\  0.232  \\  0.107  \\  0.038 \end{bmatrix}, ~~
w^{\text{U}+} = \begin{bmatrix} 0.517  \\  0.323  \\  0.131  \\  0.046 \end{bmatrix}
$"

L"$\varepsilon^\text{L} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}, ~~
\varepsilon^\text{U} = \begin{bmatrix} 0.0  \\  0.0  \\  0.0  \\  0.0 \end{bmatrix}
$"

\tilde{A}_1 = \begin{bmatrix} \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 1.41, \left[ 1.41, 1.88 \right], 2.5 \right] & \left[ 2.25, \left[ 3.56, 4.04 \right], 4.85 \right] & \left[ 8.14, \left[ 10.88, 13.34 \right], 13.47 \right] \\ \left[ 0.4, \left[ 0.53, 0.71 \right], 0.71 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 1.01, \left[ 2.12, 2.55 \right], 3.06 \right] & \left[ 3.65, \left[ 6.48, 8.4 \right], 8.48 \right] \\ \left[ 0.21, \left[ 0.25, 0.28 \right], 0.44 \right] & \left[ 0.33, \left[ 0.39, 0.47 \right], 0.99 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 1.88, \left[ 3.02, 3.34 \right], 5.33 \right] \\ \left[ 0.07, \left[ 0.07, 0.09 \right], 0.12 \right] & \left[ 0.12, \left[ 0.12, 0.15 \right], 0.27 \right] & \left[ 0.19, \left[ 0.3, 0.33 \right], 0.53 \right] & \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] \end{bmatrix}

\tilde{A}_2 = \begin{bmatrix} \left[ 1.0, \left[ 1.0, 1.0 \right], 1.0 \right] & \left[ 1.5